### 2. Implementing a Softmax Regression Model with Tensorflow on MINIST

**Dataset:**<br>
MNIST is a simple computer vision dataset.<br>
It consists of images of handwritten digits like these:
<img src="Image/MNIST.png",width=550,align="left">
It also includes labels for each image. 

**Mathematical expression of the algorithm**: 

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{6}$$


**Step 1: Import libraries**

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

Define paramaters for the model

In [2]:
learning_rate = 0.01 # common chosen value for learning rate 
batch_size = 128 #we train our model on a subset of original minist data 
n_epochs = 30 # how many times to iterate through our model 

**Step 1: Read in data** <br>
**using TF Learn's built in function to load MNIST data to the folder data/mnist**

In [3]:
mnist = input_data.read_data_sets('/data/mnist', one_hot=True) 

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


**Step 2: Create Placeholders for Features and Labels **<br>
The MNIST data is split into three parts: 55,000 data points of training data (mnist.train), 10,000 points of test data (mnist.test), and 5,000 points of validation data (mnist.validation). <br>
Each image in the MNIST data is of shape 28*28 = 784 therefore, each image is represented with a 1x784 tensor <br>
There are 10 classes for each image, corresponding to digits 0 - 9. each lable is one hot vector.

In [4]:
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder') 
Y = tf.placeholder(tf.int32, [batch_size, 10], name='Y_placeholder')

**Step 3: Create Placeholder for Weights and Bias**
- w is initialized to random variables with mean of 0, stddev of 0.01<br>
- b is initialized to 0<br>
- *you can try different initialization value for w and b<br>
- shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)<br>
- shape of b depends on Y<br>

In [5]:
w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name="bias")

**Step 4: build model**<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer

In [6]:
logits = tf.matmul(X, w) + b 

**Step 5: Define Loss Function**
use cross entropy of softmax of logits as the loss function

In [7]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch

** Step 6: define training op**
using gradient descent with learning rate of 0.01 to minimize loss

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

**Step 7: Train your model in a tensorflow session**

In [17]:
with tf.Session() as sess:
    # to visualize using TensorBoard
    #writer = tf.summary.FileWriter('./graphs/logistic_reg', sess.graph)
    
    
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs
    

    # test the model

    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(

    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
        accuracy_value = accuracy_batch[0]
        total_correct_preds =total_correct_preds+ accuracy_value
        #print('Accuracy epoch {0}: {1}'.format(i, accuracy_batch.value))
        
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

    #writer.close()

Average loss epoch 0: 0.36923096935093264
Average loss epoch 1: 0.2899195619257458
Average loss epoch 2: 0.2851277754440174
Average loss epoch 3: 0.2761288227595927
Average loss epoch 4: 0.27771557673013014
Average loss epoch 5: 0.2701860347758362
Average loss epoch 6: 0.26749655519262616
Average loss epoch 7: 0.2672456250914605
Average loss epoch 8: 0.2716234384195788
Average loss epoch 9: 0.2664784688354928
Average loss epoch 10: 0.2621698060826266
Average loss epoch 11: 0.26248418993049566
Average loss epoch 12: 0.2609330145316524
Average loss epoch 13: 0.26277542039578333
Average loss epoch 14: 0.2616173372124181
Average loss epoch 15: 0.2563137784079238
Average loss epoch 16: 0.2583873208725091
Average loss epoch 17: 0.25996593816644387
Average loss epoch 18: 0.2589261467570747
Average loss epoch 19: 0.25506325580162326
Average loss epoch 20: 0.2557765194117189
Average loss epoch 21: 0.2556589759521551
Average loss epoch 22: 0.2533363988905242
Average loss epoch 23: 0.254198067845